In [1]:
import numpy as np
import tensorflow as tf
from utils import *
from tensorflow.contrib import learn
from tensorflow.contrib import layers
from tensorflow.contrib.learn import *
from tensorflow.contrib.learn.python.learn.estimators import model_fn as model_fn_lib

tf.logging.set_verbosity(tf.logging.INFO)

## Input function for training

- Get image data from the given directory 
- Put the data into TensorFlow Queue
- Return (features, label)
    - features: a Tensor with shape (batch_size, height, width, channels) 
    - label: a Tensor with shape (batch_size)

In [2]:
train_input_fn = read_img(data_dir='data/train', batch_size=32, shuffle=True)

In [3]:
train_input_fn()

(<tf.Tensor 'features:0' shape=(?, 256, 256, 3) dtype=float32>,
 <tf.Tensor 'label:0' shape=(?,) dtype=int32>)

### Define CNN model function

In [ ]:
def cnn_model_fn(features, labels, mode, params):
    """
    Model function for CNN
    :param features: images features with shape (batch_size, height, width, channels)
    :param labels: images category with shape (batch_size)
    :param mode: Specifies if this training, evaluation or
                 prediction. See `model_fn_lib.ModeKey`
    :param params: dict of hyperparameters
    :return: predictions, loss, train_op, Optional(eval_op). See `model_fn_lib.ModelFnOps`
    """
    
    # Convolutional Layer #1
    conv1 = tf.layers.conv2d(
        inputs=features,
        filters=32,
        kernel_size=[3, 3],
        padding="same",
        activation=tf.nn.relu)

    # Pooling Layer #1
    pool1 = tf.layers.max_pooling2d(inputs=conv1, pool_size=[2, 2], strides=2)
    
    # Convolutional Layer #2 and Pooling Layer #2
    """
    Your code here
    """

    # Convolutional Layer #3 and Pooling Layer #3
    """
    Your code here
    """
    pool3 = ???
    
    # Dense Layer
    pool_flat = tf.reshape(pool3, [-1, ??? * ??? * ???])
    dense = tf.layers.dense(...)
    dropout = tf.layers.dropout(inputs=dense, rate=params['drop_out_rate']
                                , training=mode == learn.ModeKeys.TRAIN)
    
    # Logits Layer, a final layer before applying softmax
    logits = ???
    
    # Calculate Loss (for both TRAIN and EVAL modes)
    if mode != learn.ModeKeys.INFER:
        onehot_labels = tf.one_hot(indices=tf.cast(labels, tf.int32), depth=5, name="onehot")
        loss = ???
        
    # Configure the Training Op (for TRAIN mode)
    if mode == learn.ModeKeys.TRAIN:
        train_op = tf.contrib.layers.optimize_loss(...)
    
    # Generate Predictions
    predictions = {
        "classes": ???,
        "probabilities": ???
    }
    
    # Return a ModelFnOps object
    return model_fn_lib.ModelFnOps(mode=mode, 
                                   predictions=predictions, 
                                   loss=loss, 
                                   train_op=train_op)

### Define Data Preprocessing

In [ ]:
def feature_engineering_fn(features, labels):
    """
    feature_engineering_fn: Feature engineering function. Takes features and
                              labels which are the output of `input_fn` and
                              returns features and labels which will be fed
                              into `model_fn`
    """
    
    features = tf.to_float(features)
    
    # Preprocessing or Data Augmentation
    # tf.image implements most of the standard image augmentation

    # Example
    # Subtract off the mean and divide by the variance of the pixels.
    features = tf.map_fn(tf.image.per_image_standardization, features)
    
    return features, labels

### Instantiate an Estimator

In [ ]:
run_config = RunConfig(save_summary_steps=10, keep_checkpoint_max=2, save_checkpoints_secs=30)
params = ???
#use "model/plain_cnn" as model_dir
cnn_classifier = learn.Estimator(....)

In [ ]:
# Configure the accuracy metric for evaluation
metrics = {
    "accuracy":
        learn.MetricSpec(
            metric_fn=tf.metrics.accuracy, prediction_key="classes")
}

### Define input function for validation monitor and Instantiate a Validation Monitor

In [ ]:
#validation data is in 'data/validate' folder, batch size = 128
validate_input_fn = ???
validation_monitor = monitors.ValidationMonitor(input_fn=validate_input_fn,
                                                eval_steps=10,
                                                every_n_steps=50,
                                                metrics=metrics,
                                                name='validation')

### Start training the model

In [ ]:
cnn_classifier.fit(...)

### Final evaluation on unseen test data set

In [ ]:
#test data is in 'data/test' folder, batch size = 512
test_input_fn = ???
#steps = 1
cnn_classifier.evaluate(...)